In [28]:
import os
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
import json
# import tesseract
import pytesseract
from typing import *
from box import ConfigBox
from pathlib import Path
import re, sys, os
from functools import partial



import io
import os
import base64
import numpy as np
from PIL import Image
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage, AIMessage


import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever


from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

from dotenv import load_dotenv

from dataclasses import dataclass

load_dotenv()

True

In [2]:
from backend.src.utils.common import *
from backend.logger import logger
from backend.src.constants import *

In [3]:
os.getcwd()
os.chdir("../../")

In [4]:
%load_ext watermark
%watermark -a "Rajesh goldy" -vmp langchain,unstructured,openai,pydantic,pytesseract

Author: Rajesh goldy

Python implementation: CPython
Python version       : 3.9.21
IPython version      : 8.18.1

langchain   : 0.3.17
unstructured: 0.16.11
openai      : 1.61.1
pydantic    : 2.10.6
pytesseract : 0.3.13

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [31]:
from unstructured.documents.elements import CompositeElement, Table, Image, ListItem  # Import ListItem

class Extractor:
    def __init__(self):
        pass
    
    def get_metadata(raw_pdf_element: List) -> List:
        """To get the metadata of elements

        Args:
            raw_pdf_element (List): list of elements in pdf

        Returns:
            List: containing meta data
        """
        metadata = []
        for element in raw_pdf_element:
            metadata.append(element.metadata.to_dict())
        
        return metadata
    
    def add_metadata(self, element, **kwargs):
        metadata = kwargs
        return Document(element, metadata)
 
    def get_year(self, file: str) -> str:
        """To get the year from file name
        Args:
            file: name of with extension"""
        
        year = re.search(r"\d{4}", file).group()    
        if year:
            return str(year)
        else:
            try:
                file_name = os.path.split(file)[-1].split(".")[0]
                return file_name
            except Exception as e:
                logger.error(f"Error in getting year from file: {file} with error: {e}")
                return "1.0"
        
    def extract_data(self, pdf_file, **kwargs):
        raw_pdf_elements = partition_pdf(filename=pdf_file, **kwargs)
        return raw_pdf_elements
    
    def generate_unique_id(self, data: Iterator)-> List[str]:
        """To generate unique id for each element"""
        
        unique_id = []
        for _ in data:
            unique_id.append(str(uuid.uuid4()))
        
        return unique_id
    
    def generate_document(self, content: List, id_key: str):
        """To generate document from content"""
                
        ids = self.generate_unique_id(content)
        summaries = [Document(page_content=summary, metadata={id_key: ids[index]})
                         for index, summary in enumerate(content)]
        
        return summaries
    
    @staticmethod
    def seprate_data_metadata_for_text(data: List[CompositeElement]) -> Tuple[List, List]:
        """To seprate data and metadata from dict

        Args:
            data (List[dict]): List containing raw_pdf_elements
            text_name (str): key to get text from dict. Defaults to "text"
            metadata_name (str): key to get metadata from dict. Defaults to "metadata"

        Returns:
            Tuple: containing text, metadata
        """
        text, metadata = [], []
        for i in data:
            text.append(str(i))
            metadata.append(i.metadata.to_dict())
        return text, metadata
        

In [19]:

from unstructured.documents.elements import CompositeElement, Table, Image, ListItem  # Import ListItem
from unstructured.documents.elements import CompositeElement, Table, Image, ListItem  # Import ListItem



@dataclass
class DataIngestionConfig:
    raw: str # raw pdf path
    reports: str # processed reports path
    metadata: list # metadata to collect from raw_pdf_elements
    
    
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH, PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
    def get_data_ingestion_params(self) -> DataIngestionConfig:
        params = DataIngestionConfig(
            raw=self.config.data_dir.raw,
            reports=self.config.data_dir.reports,
            metadata=self.config.metadata)
        return params
    
    
class DataIngestion(Extractor):
    def __init__(self, config):
        super(DataIngestion, self).__init__()
        self.config = config
        self.raw_pdf_elements = None
        self.current_pdf_file = None # to keep track of current pdf file being processed (future case)
    
    
    def process_pdf(self, pdf_file_path: str, **kwargs):  
        """To extract the data from pdf file using unstructured library
            Args:
                pdf_file_path (str): full path of pdf file

            Returns:
                configBox: Box containing text_elements, table_elements, list_items
        """
        
        file_name = os.path.split(pdf_file_path)[-1].split(".")[0] # get the filename from file name and set it as directory name
        report_dir = os.path.join(self.config.reports, file_name) 
        create_directory(report_dir, is_extension_present=False) # create directory with file_name
        if kwargs.get("extract_image_block_output_dir", None) == None: # to create image folder in report_dir
            kwargs["extract_image_block_output_dir"] = os.path.join(report_dir, "images")
            
        raw_pdf_elements = self.extract_data(pdf_file_path, **kwargs)
        
        return raw_pdf_elements, report_dir
    
    
    

In [13]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_extractor_config = config_manager.get_data_ingestion_params()

data_extractor = DataIngestion(data_extractor_config)

2025-02-09 19:55:35,222 - root - INFO - Yaml read successfully from config.yaml
2025-02-09 19:55:35,229 - root - ERROR - FileNotFoundError: params.yaml


In [9]:
current_pdf_dir = "backend/data/reports/2023_removed"

In [ ]:
data, current_pdf_dir = data_extractor.process_pdf(pdf_file_path="backend/data/raw_pdfs/2023_removed.pdf",
                                  save=True,
                                    strategy="hi_res", # 
                                    split_pdf_page=True,  # to process each page seprately
                                    split_pdf_allow_failed=True, # continue processing even if some pages fail
                                    extract_images_in_pdf=True,
                                    infer_table_structure=True,
                                    chunking_strategy="by_title",
                                    extract_image_block_types = ["Image" , "Table"],
                                    max_characters=4000,
                                    new_after_n_chars=3800,
                                    combine_text_under_n_chars=2000
                                    )


In [14]:
text_data = data_extractor.get_text_data(data)

In [32]:

@dataclass 
class TextSummarizerConfig:
    model: str # model name used to generate summary
    text_summary_dir: str # path to save text summaries
    summarizer_prompt_dir: str
    text_summarizer_prompt: str 
    
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH, PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
    def get_text_summarizer(self) -> TextSummarizerConfig:
        summarizer_prompt_dir=read_json(self.config.prompts.summarizer_prompt_dir)
        
        params = TextSummarizerConfig(
            model=self.config.text_summarizer.model,
            text_summary_dir=self.config.text_summarizer.text_summary_dir,
            summarizer_prompt_dir=self.config.prompts.summarizer_prompt_dir,
            text_summarizer_prompt=summarizer_prompt_dir["text_summarizer_prompt"],
        )
        return params
    
class TextSummarizer(Extractor):
    def __init__(self, config, model):
        self.config = config
        self.model = model
        
    def generate_summary(self, docs: List[dict]):
        """To generate summary of the text

        Args:
            model (object): model used to generate summary of text
            docs (List[dict]): List containing dict >> text with metadata
                len of docs = len of dict containing text with metadata
                        dict -> text, metadata 
        """
                    
        raw_text, metadata = Extractor.seprate_data_metadata_for_text(docs)
        prompt = ChatPromptTemplate.from_template(self.config.text_summarizer_prompt)
        summarize_chain = {"element": lambda x: x} | prompt | self.model | StrOutputParser()
        summaries = summarize_chain.batch(raw_text, {"max_concurrency": 5})
        return raw_text, summaries, metadata
    
    def get_text_data(self, raw_pdf_elements)-> List[Any]:
        text = []
        for element in raw_pdf_elements:
            if isinstance(element, CompositeElement) or isinstance(element, ListItem):
                text.append(element)
        return text
    
    def add_metadata(self,raw_text, summaries, summary_metadata: dict=None):
        uuids = [str(uuid.uuid4()) for i in raw_text]
        
        raw_text_with_metadata = list(zip(raw_text, uuids))
        
        summaries_with_metadata = []
        for index, data in enumerate(zip(summaries, summary_metadata)):
            summary, metadata = data
            metadata["doc_id"] = uuids[index]
            summaries_with_metadata.append(Document(page_content=summary, metadata=metadata))
        
        return raw_text_with_metadata, summaries_with_metadata
        
        

In [33]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
text_summarizer_config = config_manager.get_text_summarizer()
model = ChatOpenAI(temperature=0, model="gpt-4o")
text_summarizer = TextSummarizer(text_summarizer_config, model)

2025-02-09 20:15:07,941 - root - INFO - Yaml read successfully from config.yaml


2025-02-09 20:15:07,971 - root - ERROR - FileNotFoundError: params.yaml
2025-02-09 20:15:07,982 - root - INFO - Json object read sucessfully 


In [34]:
raw_text, text_summaries, summary_metadata = text_summarizer.generate_summary(text_data)

2025-02-09 20:15:46,038 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.smith.langchain.com:443
2025-02-09 20:15:46,648 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Act as an assistant tasked with summarizing the tables and text. Give a concise summary of the table or text. Table or text chunk: Use of Non-GAAP Financial Information — This presentation includes non-GAAP financial measures, which help facilitate comparison of company operating performance across periods and with peer companies. Any historical non-GAAP measures included herein will be accompanied by a reconciliation to the nearest corresponding GAAP measure both at the end of this presentation and on our website at www.conocophillips.com/nongaap. For forward-looking non-GAAP measures, we are unable to provide a reconciliation to the most comparable GAAP financial measures beca

2025-02-09 20:16:04,093 - urllib3.connectionpool - DEBUG - https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/1.1" 202 33


In [40]:
summary_metadata[0]["uuid"] = "test"

In [41]:
summary_metadata[0]

{'filetype': 'application/pdf',
 'languages': ['eng'],
 'last_modified': '2025-02-06T12:34:29',
 'page_number': 1,
 'orig_elements': 'eJzNWduO3LgR/RWikTtGWvEmSt4n28CuvVkHRjxBHryGQZGlbsZqUdDFM41F/j1FUj0zttvGZIAeGOgHVjWLlyqe4inq7e8b6GAP/fze2c0TsmlqZrlqIdONajPBCpVpK4usrspCaa1a27abC7LZw6ytnjXa/L4x3o/W9XqGKcqdPvhlfr8Dt93NqKGyKNBmVV85O+9Qy8pSoXbwrp+D3du3SuTigqgil+8uyCpRKXIVxIoVp+TYG+XNdJhm2Ic9vHbX0L0ZtIHNf/EPCzOY2fn+ven0NL0fRt9gtyJXJaspdmhdB/NhgGj7+tUmLrXfLnob9/N2A/128y5qp/n93lvXOojeYgWTWcGyoryk7AkXT1gdrAe0fN8v+wbGsPuwiBmugyc2z/USlqLHA3kzo8OC64PJcf5LN3dx2Z8HppCgqC5opqxsMsGhzaqiwhCVRoqaS9qCOltgpMxrdH1Fk+uTyHiZ0yAyzuucn1AkgwcGp2LqsYNzuXMTGUaY0O06LAoF/9FZmMhe92gX4vHniZhlHLFF/ABj7Kc7MuCqSOtH8tz33vjXO9d1bpiI/wgjGf2y3XUHMu+A9DgZsWC0hYtoEJS4dZhvBsaerid+Gcngx7n1nfMXRPeW4PKmpfkPeozMnuyXbnZDF62X/RAWMl2gpekWDPr2gix9BxMuAWcYr9wEZBrAoH+M7nCK3s9gn5C/4cje4QZGZ4D4lvyhLH549uxX8m+YZnIJ13oiL/sZxj1YhycpLA1dy4j1XadHnBEmHBD90G+JngnLmfwjjtkvYZYf7x7uf+gx+OsjXAaHnzjknFoJUjRZIyxkorUmq1tWZYVS6LG2tFKbsx3yUuZVOMQ05+EQr6